In [ ]:
! mkdir pepper

In [ ]:
! unzip "/content/drive/MyDrive/PEPPER IMAGE SET try 1.zip" -d pepper

Archive:  /content/drive/MyDrive/PEPPER IMAGE SET try 1.zip
   creating: pepper/Resized/
   creating: pepper/Resized/BIG DOT/
  inflating: pepper/Resized/BIG DOT/IMG20210208170109.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210208172638.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210206131635.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210206131514.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210206131511.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210208170645.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210208130354.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210208172806.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210207112458.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210208171735.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210208124945.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210208172408.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210206131002_01.jpg  
  inflating: pepper/Resized/BIG DOT/IMG20210208173940.jpg  
  inflating: pepper/Resized/BIG

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install split-folders
import splitfolders

In [ ]:
splitfolders.ratio("/content/pepper/Resized", output="/content/dataset", seed=1337, ratio=(.8, .1, .1), group_prefix=None) # default values

Copying files: 1416 files [00:00, 7912.83 files/s]


In [ ]:
!nvidia-smi


Thu Feb 11 00:20:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 394.3MB 45kB/s 


In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [ ]:

# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/dataset/train'
valid_path = '/content/dataset/val'

In [ ]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 0s 0us/step


In [ ]:

# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [ ]:
# useful for getting number of output classes
folders = glob('/content/dataset/train/*')

In [ ]:
folders

['/content/dataset/train/WHITE PEN',
 '/content/dataset/train/FOLDED',
 '/content/dataset/train/HEALTHY',
 '/content/dataset/train/WILT',
 '/content/dataset/train/BIG DOT',
 '/content/dataset/train/RED DOT',
 '/content/dataset/train/VASOORI']

In [ ]:
# our layers - you can add more if you want
x = Flatten()(inception.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

#AUGMENTATION 1


In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

#AUGMENTATION 2


In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range=20,
		                              zoom_range=0.15,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.15,
                                  horizontal_flip=True,
                                  fill_mode="nearest")
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/dataset/train/',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1131 images belonging to 7 classes.


In [ ]:
labels = (training_set.class_indices)
print(labels)

{'BIG DOT': 0, 'FOLDED': 1, 'HEALTHY': 2, 'RED DOT': 3, 'VASOORI': 4, 'WHITE PEN': 5, 'WILT': 6}


In [ ]:
test_set = test_datagen.flow_from_directory('/content/dataset/val',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 139 images belonging to 7 classes.


In [ ]:
labels = (test_set.class_indices)
print(labels)

{'BIG DOT': 0, 'FOLDED': 1, 'HEALTHY': 2, 'RED DOT': 3, 'VASOORI': 4, 'WHITE PEN': 5, 'WILT': 6}


In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
36/36 [==============================] - 110s 3s/step - loss: 11.1844 - accuracy: 0.3036 - val_loss: 2.2913 - val_accuracy: 0.6403
Epoch 2/10
36/36 [==============================] - 105s 3s/step - loss: 1.8066 - accuracy: 0.6965 - val_loss: 1.7043 - val_accuracy: 0.7626
Epoch 3/10
36/36 [==============================] - 105s 3s/step - loss: 1.0246 - accuracy: 0.8093 - val_loss: 1.5704 - val_accuracy: 0.7122
Epoch 4/10
25/36 [===================>..........] - ETA: 28s - loss: 0.9504 - accuracy: 0.8058

In [ ]:
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
disease = ['BIG DOT', 'FOLDED', 'HEALTHY', 'RED DOT', 'VASOORI', 'WHITE PEN', 'WILT']
%matplotlib inline
from matplotlib import pyplot as plt
dir_path="/content/dataset/train/BIG DOT/"
k=0
count=0
for i in os.listdir(dir_path):
  k=k+1
  img = image.load_img(dir_path+i, target_size=(224, 224))
  # Preprocessing the image
  x = image.img_to_array(img)
  # x = np.true_divide(x, 255)
  ## Scaling
  x=x/255
  x = np.expand_dims(x, axis=0)
  preds = model.predict(x)
  if disease[np.argmax(preds)] =='BIG DOT':
    count=count+1
  #print(disease[np.argmax(preds)])
print(k)
print(count)
print(count/k)

190
161
0.8473684210526315


In [ ]:
from keras.models import load_model

model.save('/content/drive/MyDrive/ResNetTomato/PepperResNet50.h5')  

In [ ]:
import os
from google.colab.patches import cv2_imshow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
disease= ['BIG DOT', 'FOLDED', 'HEALTHY', 'RED DOT', 'VASOORI', 'WHITE PEN', 'WILT']
%matplotlib inline
from matplotlib import pyplot as plt

import matplotlib.image as mpimg
#img = mpimg.imread('your_image.png')

dir_path="/content/dataset/val/"
k=0
count=0
for y in range(0, 6):
  for i in os.listdir(dir_path+disease[y]):
    k=k+1
    img = image.load_img(dir_path+disease[y]+"/"+i, target_size=(224, 224))
    imgplot = plt.imshow(img)
    plt.show()
    # Preprocessing the image
    x = image.img_to_array(img)
    # x = np.true_divide(x, 255)
    ## Scaling
    x=x/255
    x = np.expand_dims(x, axis=0)
    preds = model.predict(x)
    if disease[np.argmax(preds)] ==str(disease[y]):
      count=count+1
    print(disease[np.argmax(preds)])
  print(k)
  print(count)
  print(count/k)

In [ ]:
import os

disease= ['BIG DOT', 'FOLDED', 'HEALTHY', 'RED DOT', 'VASOORI', 'WHITE PEN', 'WILT']

dir_path="/content/dataset/val/"
k=0
for y in range(0, 6):
  print(disease[y])
  for i in os.listdir(dir_path+disease[y]):
    k=k+1
  print(k)


BIG DOT
23
FOLDED
44
HEALTHY
72
RED DOT
103
VASOORI
111
WHITE PEN
129
